In [1]:
import tensorflow as tf
#from tensorflow.examples.tutorials.mnist import input_data
import tensor_mnist.mnist as input_data

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
max_steps = 1000
learning_rate = 0.001
dropout = 0.9
data_dir = './MNIST_data_bak'
log_dir = './logs/mnist_with_summaries'

In [3]:
mnist = input_data.read_data_sets(data_dir, one_hot=True)
sess = tf.InteractiveSession()

filepath:./MNIST_data_bak\train-images-idx3-ubyte.gz
Extracting ./MNIST_data_bak\train-images-idx3-ubyte.gz
filepath:./MNIST_data_bak\train-labels-idx1-ubyte.gz
Extracting ./MNIST_data_bak\train-labels-idx1-ubyte.gz
filepath:./MNIST_data_bak\t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data_bak\t10k-images-idx3-ubyte.gz
filepath:./MNIST_data_bak\t10k-labels-idx1-ubyte.gz
Extracting ./MNIST_data_bak\t10k-labels-idx1-ubyte.gz


输入层的定义

In [4]:
with tf.name_scope('input'):
    # x,28*28列
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')

with tf.name_scope('input_reshape'):
    # 784维度变形为图片保持到节点
    # -1 代表进来的图片的数量、28，28是图片的高和宽，1是图片的颜色通道
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    # summary.image把想要输出的图片存到log_dir里，10表示画十张图片
    tf.summary.image('input', image_shaped_input, 10)

权重的初始化

In [5]:
# 定义神经网络的初始化方法
# 定义变量，在运行过程中会改变的变量
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [6]:
# 通过tf.summary这个方法往logs文件中写数据

# 定义Variable变量的数据汇总函数，我们计算出变量的mean、stddev、max、min
# 对这些标量数据使用tf.summary.scalar进行记录和汇总
# 使用tf.summary.histogram直接记录变量var的直方图数据
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        # scalar生成一些数值
        # 把均值保存到log_dir
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        # 存方差，最大值，最小值，画柱状图
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [7]:
# 设计一个MLP多层神经网络来训练数据
# 在每一层中都对模型数据进行汇总
def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            # 初始化w参数
            weights = weight_variable([input_dim, output_dim])
            # 把w参数保存在图中
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)
        with tf.name_scope('Wx_plus_b'):
            # 正向传播
            preactivate = tf.matmul(input_tensor, weights) + biases
            tf.summary.histogram('pre_activations', preactivate)
        # act:定义激活函数
        activations = act(preactivate, name='activation')
        # 存为柱状图
        tf.summary.histogram('activations', activations)
        return activations

In [8]:
# 我们使用刚刚定义的函数创建一层神经网络，输入维度是图片的尺寸784=28*28
# 输出的维度是隐藏节点数500，再创建一个Dropout层，并使用tf.summary.scalar记录keep_prob
# 然后使用nn_layer定义神经网络输出层，其输入维度为上一层隐含节点数500，输出维度为类别数10
# 同时激活函数为全等映射identity(tf.identity:它返回一个和输入的 tensor 大小和数值都一样的 tensor ,类似于 y=x 操作)，暂时不使用softmax

# 构建第一个隐层
hidden1 = nn_layer(x, 784, 500, 'layer1')

# 定义droupout层，并保存
with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('dropout_keep_probability', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

y = nn_layer(dropped, 500, 10, 'layer2', act=tf.identity)

# 使用tf.nn.softmax_cross_entropy_with_logits()对前面的输出层的结果进行Softmax
# 处理并计算交叉熵损失cross_entropy，计算平均的损失，使用tf.summary.scalar进行统计汇总
with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_)
    with tf.name_scope('total'):
        cross_entropy = tf.reduce_mean(diff)
# 保存交叉熵
tf.summary.scalar('cross_entropy', cross_entropy)


# 下面使用Adam优化器对损失进行优化，同时统计预测正确的样本数并计算正确率accuracy，汇总
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar('accuracy', accuracy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

In [9]:
# 因为我们之前定义了太多的tf.summary汇总操作，逐一执行这些操作太麻烦，
# 使用tf.summary.merge_all()直接获取所有汇总操作，以便后面执行
merged = tf.summary.merge_all()
# 定义两个tf.summary.FileWriter文件记录器再不同的子目录，分别用来存储训练和测试的日志数据
train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
test_writer = tf.summary.FileWriter(log_dir + '/test')
# 同时，将Session计算图sess.graph加入训练过程，这样再TensorBoard的GRAPHS窗口中就能展示
# 整个计算图的可视化效果，最后初始化全部变量
tf.global_variables_initializer().run()

In [13]:
# 可视化：
#打开prompt命令窗口，切换到工作目录 输入： tensorboard --logdir ./logs/mnist_with_summaries/
# 注意路径不能有中文，要不然运行不通，tensorboard找不到数据。

In [ ]:
# tf.InteractiveSession():它能让你在运行图的时候，插入一些计算图，这些计算图是由某些操作(operations)构成的。
# 这对于工作在交互式环境中的人们来说非常便利，比如使用IPython。
# tf.Session():需要在启动session之前构建整个计算图，然后启动该计算图。